In [ ]:
%pwd

In [ ]:
## Loading Imporatant Libraries:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df_country=pd.read_csv('/kaggle/input/india-odi-history/countries_list.csv')
df_matches=pd.read_csv('/kaggle/input/india-odi-history/file_cricket.csv')


In [ ]:
df_country
df_matches

In [ ]:
df_matches.info()

### Here we are adding  India column  which shows match results such as  won, lost and draw.

In [ ]:
condtions=[(df_matches['result'].str.contains('India won')),(df_matches['result'].str.contains('No Result'))
           ,(df_matches['result'].str.contains('Match Tied')),
            ~(df_matches['result'].str.contains('India won'))]
values=['Won','No result','Tied','Lost']                                                             

In [ ]:
df_matches['India']=np.select(condtions,values)

In [ ]:
df_matches['India'].value_counts()

In [ ]:
print('India play total no of matches =',df_matches['India'].value_counts().sum())
print('India won total no of matches =',df_matches['India'].value_counts()[0])
print('India loss total no of matches =',df_matches['India'].value_counts()[1])
print('No Results matches =',df_matches['India'].value_counts()[2])
print('Drwan matches =',df_matches['India'].value_counts()[3])

In [ ]:
sns.countplot(df_matches['India']);
plt.xlabel('Match result')
plt.ylabel('No of Matchs')

###  Lets Add column showing that who win by batting first:


In [ ]:
df_matches['Country_won']=df_matches['result'].str.split('won',expand=True)[0]

In [ ]:
df_matches['result']

In [ ]:
conditions=[(df_matches['result'].str.contains('wickets')),(df_matches['result'].str.contains('No Result')),
            (df_matches['result'].str.contains('Match')),
           ~(df_matches['result'].str.contains('wickets'))]
values=['Bowl_First','None','Drawn','Bat_First']
df_matches['Bat_or_Bowl_First']=np.select(conditions,values)

In [ ]:
df_matches['Bat_or_Bowl_First'].value_counts()

In [ ]:
df_bat_first=df_matches[df_matches['Bat_or_Bowl_First']=='Bat_First']['Country_won'].value_counts().to_frame()
df_bat_first.reset_index(inplace=True)
df_bat_first=df_bat_first.rename(columns={'index':'Country','Country_won':'bat_win'})
df_bat_first=df_bat_first.sort_values(by='bat_win',ascending=False)


In [ ]:
df_bowl_first=df_matches[df_matches['Bat_or_Bowl_First']=='Bowl_First']['Country_won'].value_counts().to_frame()
df_bowl_first.reset_index(inplace=True)
df_bowl_first=df_bowl_first.rename(columns={'index':'Country','Country_won':'bowl_win'})
df_bowl_first=df_bowl_first.sort_values(by='bowl_win',ascending=False)
df_bowl_first
df_bat_bowl_win=pd.merge(df_bat_first,df_bowl_first)
df_bat_bowl_win['Total_win']=df_bat_bowl_win['bat_win']+df_bat_bowl_win['bowl_win']
df_bat_bowl_win

In [ ]:
df_matches['Country_won']=df_matches['Country_won'].str.strip()
df_matches['Country_won'].value_counts()

###  Which Ground managed to produce maximum wins to India?

In [ ]:
df_matches[df_matches['Country_won']=='India']['Ground'].value_counts()[:6].to_frame()

*Sharjah Cricket Stadium is most favourable stadium for team india.*

###  Which month/Year India has maximum number of wins?

In [ ]:
## Here we need to extract month and year from Date column:
df_matches['Date']=pd.to_datetime(df_matches['Date'])

In [ ]:
df_matches['Year']=df_matches['Date'].dt.year
df_matches['Month']=df_matches['Date'].dt.month
df_matches.drop(columns=['Unnamed: 0','Index'],inplace=True)

In [ ]:
df_matches

In [ ]:
top_year=df_matches[df_matches['Country_won']=='India']['Year'].value_counts()[:5].to_frame()
top_year.reset_index(inplace=True)
top_year.columns=['Year','Win']
top_year

In [ ]:
total_match = df_matches['Year'].value_counts().to_frame()
total_match.reset_index(inplace=True)
total_match.columns=['Year','total_match']
total_match
pd.merge(top_year,total_match)

In [ ]:
df_matches[df_matches['Country_won']=='India']['Month'].value_counts()[:5]

**1998 is the Year and October is the month In which team india won most.**  

###  Which foreign country has maximum wins against India?


In [ ]:
df_matches[df_matches['Country_won']!='India']['Country_won'].value_counts()[:5].to_frame()

###  Which ground has max winning chances if India wants to play against Australia/ England?


In [ ]:
import re
temp_c=df_matches.copy()
temp_c['Country_Against']= temp_c['Series'].apply(lambda x : re.sub(r'.*\(v.\s|\)|\d+\-\d+\s|(\sv.\sIndia|India\sv.\s)|\d+\s|','',x))
temp_c['Country_Against'].unique()
temp_c=temp_c[['Ground','Country_Against','Country_won']]

In [ ]:
temp_c

In [ ]:
fav_stadium=temp_c[(temp_c['Country_Against']=='Australia') & (temp_c['Country_won']=='India')]['Ground'].value_counts().to_frame()
fav_stadium.reset_index(inplace=True)
fav_stadium.columns=['Ground','Win']
stadium=temp_c[(temp_c['Country_Against']=='Australia')]['Ground'].value_counts().to_frame()
stadium.reset_index(inplace=True)
stadium.columns=['Ground','Total_match']
most_win_sta=pd.merge(fav_stadium,stadium)
most_win_sta['Winning%']=most_win_sta['Win']/most_win_sta['Total_match']*100
most_win_sta=most_win_sta.sort_values(by=['Win'],ascending=False)
most_win_sta[:10]

In [ ]:
fav_stadium=temp_c[(temp_c['Country_Against']=='England') & (temp_c['Country_won']=='India')]['Ground'].value_counts().to_frame()
fav_stadium.reset_index(inplace=True)
fav_stadium.columns=['Ground','Win']
stadium=temp_c[(temp_c['Country_Against']=='England')]['Ground'].value_counts().to_frame()
stadium.reset_index(inplace=True)
stadium.columns=['Ground','Total_match']
most_win_sta=pd.merge(fav_stadium,stadium)
most_win_sta['Winning%']=most_win_sta['Win']/most_win_sta['Total_match']*100
most_win_sta=most_win_sta.sort_values(by=['Win'],ascending=False)
most_win_sta[:13]

### How did India Perform in ODI after 2011 world cup?

In [ ]:
after_wc_df = df_matches[df_matches['Date'] > '2011-04-02']

In [ ]:
win_af_wc=after_wc_df[after_wc_df['India']=='Won']['Year'].value_counts().to_frame()
win_af_wc.reset_index(inplace=True)
win_af_wc.columns=['Year','Win']
win_af_wc
loss_af_wc=after_wc_df[after_wc_df['India']=='Lost']['Year'].value_counts().to_frame()
loss_af_wc.reset_index(inplace=True)
loss_af_wc.columns=['Year','Lost']
win_af_wc
win_loss_df=pd.merge(win_af_wc,loss_af_wc)
win_loss_df=win_loss_df.sort_values(by=['Year'])
win_loss_df

###  Which 5 year Period India saw huge win percent?


In [ ]:
df_india=df_matches[df_matches['Country_won']=='India']['Year'].value_counts().to_frame()

In [ ]:
df_india.reset_index(inplace=True)
df_india=df_india.rename(columns={'index':'Year','Year':'Count'})


In [ ]:
df_india['Mean']=df_india['Count']/(df_india['Count'].sum())
df_india=df_india.sort_values(by='Year',ascending=False)
df_india